In [1]:
import anyjson as json
import requests
from bs4 import BeautifulSoup
#import scalpyr
import urllib
from json import load 
import pandas as pd
from pandas import DataFrame
import numpy as np
import codecs
#from urllib import request
from urllib2 import urlopen # python 2.7 version
#import urlopen
from json import load
#import sqlite3
#import csv
#import codecs
#import cStringIO
#import sys
import hashlib

In [ ]:
# Seatgeek event table

# Create events table
evs = pd.DataFrame()

row_iterator = pfs.iterrows()
#_, last = row_iterator.next()  # take first item from row_iterator
for i, pf in row_iterator:
#if True:
    #print pf
    pid = int(pf['id'])
    #print pid

    # slug
    idurl = 'http://api.seatgeek.com/2/events?performers.id='

    try:
        url = idurl 
        search_term = str(pid)
                
        url += search_term
        print url
        
        #response = request.urlopen(url)
        response = urlopen(url)
        json_obj = load(reader(response))

    except ValueError:
        pass
        #print (ArtistList['artist_name'][a] )
        
    #print json_obj
    
    events = json_obj['events']
    # add list of event ids to performer
    pfs.loc[pid,'events'] = ','.join([ str(e['id']) for e in events ])

    for event in events:
        ecols = []
        efields = []

        # id type and title (and score)
        l = ['id','type','title','short_title','score','url']
        ecols += l
        efields += [event[s] for s in l ]

        # dates and times
        l = ['time_tbd','date_tbd','datetime_local','datetime_utc','created_at']
        ecols += l
        efields += [event[s] for s in l ]

        # performers
        l = ['query_performer','num_performers','performer_ids','performer_slugs']
        perfs = event['performers']
        num_perfs = len(perfs)
        perf_ids = [p['id'] for p in perfs]
        perf_slugs = [p['slug'] for p in perfs]
        ecols += l
        efields += [pid,num_perfs,perf_ids,perf_ids]

        # venue
        l = ['venue_id','venue_slug']
        venue = event['venue']
        venue_id = venue['id'] 
        venue_slug = venue['slug']
        ecols += l
        efields += [venue_id,venue_slug]

        # stats
        stats = event['stats']
        ecols += stats.keys()
        efields += [stats[key] for key in stats.keys()]

        # make dataframe
        ev = pd.DataFrame([efields],columns=ecols, index=[event['id']])
        evs = evs.append(ev)

evs.head()    

In [4]:
print len(json_obj['resultsPage']['results']['event'])
event = json_obj['resultsPage']['results']['event'][84]
event.keys()

85


[u'status',
 u'popularity',
 u'displayName',
 u'ageRestriction',
 u'venue',
 u'uri',
 u'start',
 u'location',
 u'performance',
 u'type',
 u'id']

In [5]:
event['venue'].keys()

[u'displayName', u'uri', u'lat', u'metroArea', u'lng', u'id']

In [7]:
event['performance'][0]

{u'artist': {u'displayName': u'Zac Brown Band',
  u'id': 514278,
  u'identifier': [{u'href': u'http://api.songkick.com/api/3.0/artists/mbid:507226f4-0842-4f43-950e-81a7017230ec.json',
    u'mbid': u'507226f4-0842-4f43-950e-81a7017230ec'}],
  u'uri': u'http://www.songkick.com/artists/514278-zac-brown-band?utm_source=35442&utm_medium=partner'},
 u'billing': u'headline',
 u'billingIndex': 1,
 u'displayName': u'Zac Brown Band',
 u'id': 45781813}

In [65]:
mbid = '507226f4-0842-4f43-950e-81a7017230ec'
api_key = 'CVpeRf49J3duiKg7'
min_date = '2010-01-01'
max_date = '2015-07-27'
per_page = 100
page = 6



url = 'http://api.songkick.com/api/3.0/artists/mbid:'+mbid+'/gigography.json?apikey='+api_key
url = url+'&min_date='+min_date+'&max_date='+max_date+'&per_page='+str(per_page)+'&page='+str(page)
print url

reader = codecs.getreader("utf-8")

response = urlopen(url)
json_obj = load(response)

print json_obj['resultsPage'].keys()
print json_obj['resultsPage']['totalEntries']
#print json_obj['resultsPage']['perPage']
print json_obj['resultsPage']['page']
print len(json_obj['resultsPage']['results']['event'])


http://api.songkick.com/api/3.0/artists/mbid:507226f4-0842-4f43-950e-81a7017230ec/gigography.json?apikey=CVpeRf49J3duiKg7&min_date=2010-01-01&max_date=2015-07-27&per_page=100&page=6
[u'status', u'totalEntries', u'perPage', u'results', u'page']
485
6


KeyError: 'event'

In [19]:
print len(json_obj['resultsPage']['results']['event'])
event = json_obj['resultsPage']['results']['event'][84]
print event.keys()
print event['venue'].keys()
print event['start'].keys()
print event['location'].keys()
print event['performance'][0].keys()
print event['displayName']
print event['type']

85
[u'status', u'popularity', u'displayName', u'ageRestriction', u'venue', u'uri', u'start', u'location', u'performance', u'type', u'id']
[u'displayName', u'uri', u'lat', u'metroArea', u'lng', u'id']
[u'date', u'datetime', u'time']
[u'lat', u'city', u'lng']
[u'artist', u'displayName', u'id', u'billing', u'billingIndex']
Zac Brown Band at Amphitheater Northwest (July 12, 2015)
Concert


In [71]:
artists.head()

,artist_name,artist_name_clean,artist_name_dash,artist_name_query,artist_name_underscore,sg_artist_id,event_count,genres,sg_type,sg_artist_score,...,isni_list,country,area,begin_area,life_span_begin,genre1,genre2,genre3,genre4,genre5
0,Zach Sobiech,Zach Sobiech,zach-sobiech,zach+sobiech,zach_sobiech,160111,0,NaN,band,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Soulja Boy Tell'em,Soulja Boy Tell'em,soulja-boy-tell'em,soulja+boy+tell'em,soulja_boy_tell'em,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Austin Mahone,Austin Mahone,austin-mahone,austin+mahone,austin_mahone,12596,2,NaN,band,0.66274,...,NaN,US,United States,San Antonio,1996-04-04T00:00:00,RB,Pop,NaN,NaN,NaN
3,Britney Spears,Britney Spears,britney-spears,britney+spears,britney_spears,376,39,pop,band,0.68469,...,0000000109898265,US,United States,McComb,1981-12-02T00:00:00,Pop,NaN,NaN,NaN,NaN
4,Diggy,Diggy,diggy,diggy,diggy,28294,0,hip-hop,band,0.31305,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
artists = pd.DataFrame.from_csv('../mode/mode_artists_left_join.csv',index_col=None)
artist_list = artists[pd.notnull(artists.mbz_id)][['artist_name_clean','mbz_id']]

In [126]:
reader = codecs.getreader("utf-8")
events = pd.DataFrame()

api_key = 'CVpeRf49J3duiKg7'
min_date = '2010-01-01'
max_date = '2015-07-27'
per_page = 100

for a in range(len(artist_list)):


    artist_name = artist_list.iloc[a]['artist_name_clean']
    mbid = artist_list.iloc[a]['mbz_id']
    
    artist_events = pd.DataFrame()
    #mbid = '507226f4-0842-4f43-950e-81a7017230ec'

    page = 1
    more_results = True
    
    print artist_name
    print a+1,'/',len(artist_list), 'artists'

    while more_results:

        url = 'http://api.songkick.com/api/3.0/artists/mbid:'+mbid+'/gigography.json?apikey='+api_key
        url = url+'&min_date='+min_date+'&max_date='+max_date+'&per_page='+str(per_page)+'&page='+str(page)
        #print url


        try:
            response = urlopen(url)
            json_obj = load(response)

            num_response = len(json_obj['resultsPage']['results']['event'])
            total_entries = json_obj['resultsPage']['totalEntries']

            print per_page*(page-1)+num_response, '/', total_entries, 'events'

        except:
            more_results = False

        else:

            if num_response < per_page:
                more_results = False
            else:
                page += 1

            for event in json_obj['resultsPage']['results']['event']:
                l = []
                col = []

                # get basic info
                c_bas = ['id','mbz_id','artist_name','event_name','type','popularity']
                col+=c_bas
                try:
                    l_bas = [event['id'],mbid,artist_name,event['displayName'],event['type'],event['popularity']]
                    l += l_bas
                except:
                    l += ['' for i in range(len(c_bas))]

                # get location information
                c_loc = [ u'city',u'lat', u'lng']
                col += c_loc
                try:
                    l_loc = [ event['location'][key] for key in c_loc ]
                    l += l_loc
                except:
                    l += ['' for i in range(len(c_loc))]

                # get venue information
                c_ven = ['venue_name','country','state']
                col += c_ven
                try:
                    ven = event['venue']
                    met = ven['metroArea']
                    l_ven = [ ven['displayName'],met['country']['displayName'],met['state']['displayName']]
                    l += l_ven
                except:
                    l += ['' for i in range(len(c_ven))]

                # get billing info
                c_bill = ['artist_id','billing','billingIndex']
                col += c_bill
                try:
                    l_bill = ['' for i in range(len(c_bill))]
                    for i in range(len(event['performance'])):
                        if event['performance'][0]['artist']['identifier'][0]['mbid'] == mbid:
                            l_bill = [event['performance'][0][x] for x in ['id','billing','billingIndex']]
                    l += l_bill
                except:
                    l += ['' for i in range(len(c_bill))]

                # get time
                #c_time = event['start'].keys()
                c_time = [u'date', u'datetime', u'time']
                col += c_time
                try:
                    l_time = [ event['start'][key] for key in c_time ]
                    l += l_time
                except:
                    l += ['' for i in range(len(c_time))]

                #print len(l)
                #print len(col)
                # update dataframe
                df = pd.DataFrame([l],columns=col)
                df = df.set_index('id')
                #print df
                artist_events = artist_events.append(df)
                #artist_events = pd.concat([artist_events,df],axis=0, ignore_index=True)

    #artist_events
    events = events.append(artist_events)
    
events.head()

Austin Mahone
1 / 490 artists
100 / 158 events
158 / 158 events
Britney Spears
2 / 490 artists
100 / 210 events
200 / 210 events
210 / 210 events
Childish Gambino
3 / 490 artists
100 / 210 events
200 / 210 events
210 / 210 events
Glen Campbell
4 / 490 artists
100 / 195 events
195 / 195 events
Mary Mary
5 / 490 artists
87 / 87 events
Lucy Hale
6 / 490 artists
9 / 9 events
MKTO
7 / 490 artists
100 / 127 events
127 / 127 events
Tinashe
8 / 490 artists
Sean Kingston
9 / 490 artists
100 / 158 events
158 / 158 events
Keith Urban
10 / 490 artists
100 / 305 events
200 / 305 events
300 / 305 events
305 / 305 events
Juliet Simms
11 / 490 artists
23 / 23 events
Ed Sheeran
12 / 490 artists
100 / 608 events
200 / 608 events
300 / 608 events
400 / 608 events
500 / 608 events
600 / 608 events
608 / 608 events
Victorious Cast
13 / 490 artists
Akon
14 / 490 artists
62 / 62 events
Missy Elliott
15 / 490 artists
24 / 24 events
Prince Royce
16 / 490 artists
100 / 167 events
167 / 167 events
Owl City
17 / 

,mbz_id,artist_name,event_name,type,popularity,city,lat,lng,venue_name,country,state,artist_id,billing,billingIndex,date,datetime,time
id,,,,,,,,,,,,,,,,,
10327278,b69b9854-f568-4a50-b076-309c459946a8,Austin Mahone,Austin Mahone at Texas Outback Bar & Grill (Se...,Concert,0.013323,"Helsinki, Finland",60.1756,24.9342,,,,19885623,headline,1,2011-09-14,None,None
10329148,b69b9854-f568-4a50-b076-309c459946a8,Austin Mahone,Austin Mahone at Canada Games Arena (September...,Concert,0.013323,"Grande Prairie, AB, Canada",55.1712,-118.795,Canada Games Arena,Canada,AB,19890008,headline,1,2011-09-15,None,None
10327283,b69b9854-f568-4a50-b076-309c459946a8,Austin Mahone,"Austin Mahone at Canada (September 16, 2011)",Concert,0.013323,"Toronto, ON, Canada",43.6667,-79.4167,Canada,Canada,ON,19885628,headline,1,2011-09-16,None,None
10329923,b69b9854-f568-4a50-b076-309c459946a8,Austin Mahone,"Austin Mahone at New York (September 16, 2011)",Concert,0.013323,"Augusta, GA, US",33.4574,-81.995,New York,US,GA,19891683,headline,1,2011-09-16,None,None
10368608,b69b9854-f568-4a50-b076-309c459946a8,Austin Mahone,Austin Mahone at The Standard Hotel - New York...,Concert,0.013323,"New York, NY, US",40.74088,-74.00782,The Standard Hotel - New York City,US,NY,19978103,headline,1,2011-09-29,None,None


In [128]:
events.date = pd.to_datetime(events.date)
#events.to_csv('songkick_events_test.csv',index_label='id',encoding="utf-8")
events.to_csv('songkick_events_2010_15.csv',index_label='id',encoding="utf-8")

In [125]:
#events[events.artist_name =='Childish Gambino']

In [122]:
#events[events.artist_name =='Britney Spears']